In [1]:
import platform
import matplotlib.pyplot as plt

try:
    system_name = platform.system()
    
    if system_name == 'Darwin':  # macOS
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':  # Windows
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:  # Linux (예: Ubuntu, Colab)
        plt.rcParams['font.family'] = 'NanumGothic'  # or 'DejaVu Sans'
    
    # 마이너스 부호 깨짐 방지
    plt.rcParams['axes.unicode_minus'] = False

except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")
    print("그래프의 한글이 깨질 수 있습니다.")

# 0. 데이터 불러오기

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [103]:
begin_inventory = pd.read_csv('../data/drinksight-2024-begin-inventory.csv')
end_inventory = pd.read_csv('../data/drinksight-2024-end-inventory.csv')
purchase_prices = pd.read_csv('../data/drinksight-2024-purchase-prices.csv')
purchases = pd.read_csv('../data/drinksight-2024-purchases.csv')
sales = pd.read_csv('../data/drinksight-2024-sales.csv')
vendor_invoice = pd.read_csv('../data/drinksight-2024-vendor-invoice.csv')

# 1. 데이터 전처리

## 1-1. begin_inventory, end_inventory

### 1-1-1. EDA

In [104]:
print(begin_inventory.shape)
print(end_inventory.shape)

(206529, 9)
(224489, 9)


➡️ 전체적으로 inventory가 증가했음. (기초재고 inventory보다 기말재고 inventory가 더 많음)

In [105]:
begin_inventory.head()

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,begin_date
0,1_SPARROW_58,1,Sparrow,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_SPARROW_60,1,Sparrow,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_SPARROW_62,1,Sparrow,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_SPARROW_63,1,Sparrow,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_SPARROW_72,1,Sparrow,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


In [106]:
begin_inventory.isna().sum()

inventory_id    0
store_id        0
city            0
brand_id        0
item            0
size            0
on_hand         0
price           0
begin_date      0
dtype: int64

In [107]:
end_inventory.head()

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,end_date
0,1_SPARROW_58,1,Sparrow,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_SPARROW_62,1,Sparrow,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_SPARROW_63,1,Sparrow,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_SPARROW_72,1,Sparrow,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_SPARROW_75,1,Sparrow,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


In [123]:
기초인벤토리=begin_inventory['inventory_id'].nunique()
기말인벤토리=end_inventory['inventory_id'].nunique()
(기말인벤토리-기초인벤토리)/기초인벤토리

0.08696115315524697

In [131]:
기초브랜드수 = (begin_inventory['brand_id'].nunique())
기말브랜드수 = (end_inventory['brand_id'].nunique())
(기말브랜드수-기초브랜드수)/기초브랜드수

0.19261181121818632

In [124]:
기초재고량 = begin_inventory['on_hand'].sum()
기말재고량 = end_inventory['on_hand'].sum()
(기말재고량-기초재고량)/기초재고량


0.1579657642604476

In [125]:
기초재고자산 = (begin_inventory['on_hand']*begin_inventory['price']).sum()
기말재고자산 = (end_inventory['on_hand']*end_inventory['price']).sum()
(기말재고자산-기초재고자산)/기초재고자산

0.17120387627102174

#### 인벤토리 다양성은 8.7% 증가, 브랜드 다양성은 19.3% 증가, 재고량은 15.8% 증가, 재고자산은 17.1% 증가.
#### 즉, 재고과잉 증가에 영향을 준 특정 브랜드가 있을 것이다!

### 1-1-2. city 결측값 처리

In [108]:
end_inventory.isna().sum()

inventory_id       0
store_id           0
city            1284
brand_id           0
item               0
size               0
on_hand            0
price              0
end_date           0
dtype: int64

In [9]:
# city가 NaN인 행만 추출
nan_city_df = end_inventory[end_inventory['city'].isna()]
nan_city_df

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,end_date
113895,46__58,46,NaN,58,Gekkeikan Black & Gold Sake,750mL,0,12.99,2024-12-31
113896,46__62,46,NaN,62,Herradura Silver Tequila,750mL,0,36.99,2024-12-31
113897,46__63,46,NaN,63,Herradura Reposado Tequila,750mL,0,38.99,2024-12-31
113898,46__77,46,NaN,77,Three Olives Espresso Vodka,750mL,0,14.99,2024-12-31
113899,46__106,46,NaN,106,Mr Boston Peach Schnapps,Liter,0,4.49,2024-12-31
...,...,...,...,...,...,...,...,...,...
115174,46__46447,46,NaN,46447,Gascon Malbec Mendoza,750mL,0,10.99,2024-12-31
115175,46__46458,46,NaN,46458,Layer Cake Barosa Shiraz,750mL,0,15.99,2024-12-31
115176,46__46476,46,NaN,46476,Tilia Malbec Mendoza,750mL,0,9.99,2024-12-31
115177,46__46764,46,NaN,46764,Clayhouse Adobe Red Paso Rbl,750mL,0,11.99,2024-12-31


In [10]:
nan_city_df['store_id'].unique()

array([46])

In [11]:
find_city = begin_inventory[begin_inventory['store_id']==46]
find_city

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,begin_date
108343,46_CANARY_58,46,Canary,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
108344,46_CANARY_62,46,Canary,62,Herradura Silver Tequila,750mL,2,36.99,2024-01-01
108345,46_CANARY_63,46,Canary,63,Herradura Reposado Tequila,750mL,4,38.99,2024-01-01
108346,46_CANARY_77,46,Canary,77,Three Olives Espresso Vodka,750mL,4,14.99,2024-01-01
108347,46_CANARY_79,46,Canary,79,Three Olives Loopy Vodka,750mL,12,14.99,2024-01-01
...,...,...,...,...,...,...,...,...,...
110218,46_CANARY_46763,46,Canary,46763,Clayhouse Svgn Bl Paso Roble,750mL,5,11.99,2024-01-01
110219,46_CANARY_46764,46,Canary,46764,Clayhouse Adobe Red Paso Rbl,750mL,8,11.99,2024-01-01
110220,46_CANARY_46826,46,Canary,46826,LaBelle Winery Grnt St Apple,375mL,5,14.99,2024-01-01
110221,46_CANARY_46829,46,Canary,46829,Pacific Rim Yakima Gewurz,750mL,7,11.99,2024-01-01


In [12]:
check_city = begin_inventory[begin_inventory['city']=='Canary']
check_city['store_id'].unique()

array([46])

#### ➡️ end inventory의 결측치를 'canary'로 채우기

In [13]:
end_inventory["city"].fillna("Canary", inplace=True)

In [14]:
end_inventory[end_inventory['city'].isna()]

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,end_date


In [15]:
end_inventory[end_inventory['city']=="Canary"]

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,end_date
113895,46__58,46,Canary,58,Gekkeikan Black & Gold Sake,750mL,0,12.99,2024-12-31
113896,46__62,46,Canary,62,Herradura Silver Tequila,750mL,0,36.99,2024-12-31
113897,46__63,46,Canary,63,Herradura Reposado Tequila,750mL,0,38.99,2024-12-31
113898,46__77,46,Canary,77,Three Olives Espresso Vodka,750mL,0,14.99,2024-12-31
113899,46__106,46,Canary,106,Mr Boston Peach Schnapps,Liter,0,4.49,2024-12-31
...,...,...,...,...,...,...,...,...,...
115174,46__46447,46,Canary,46447,Gascon Malbec Mendoza,750mL,0,10.99,2024-12-31
115175,46__46458,46,Canary,46458,Layer Cake Barosa Shiraz,750mL,0,15.99,2024-12-31
115176,46__46476,46,Canary,46476,Tilia Malbec Mendoza,750mL,0,9.99,2024-12-31
115177,46__46764,46,Canary,46764,Clayhouse Adobe Red Paso Rbl,750mL,0,11.99,2024-12-31


In [16]:
end_inventory['inventory_id'] = (
    end_inventory['store_id'].astype(str)
    + '_'
    + end_inventory['city'].astype(str).str.upper()
    + '_'
    + end_inventory['brand_id'].astype(str)
)


In [17]:
end_inventory[end_inventory['city']=="Canary"]

,inventory_id,store_id,city,brand_id,item,size,on_hand,price,end_date
113895,46_CANARY_58,46,Canary,58,Gekkeikan Black & Gold Sake,750mL,0,12.99,2024-12-31
113896,46_CANARY_62,46,Canary,62,Herradura Silver Tequila,750mL,0,36.99,2024-12-31
113897,46_CANARY_63,46,Canary,63,Herradura Reposado Tequila,750mL,0,38.99,2024-12-31
113898,46_CANARY_77,46,Canary,77,Three Olives Espresso Vodka,750mL,0,14.99,2024-12-31
113899,46_CANARY_106,46,Canary,106,Mr Boston Peach Schnapps,Liter,0,4.49,2024-12-31
...,...,...,...,...,...,...,...,...,...
115174,46_CANARY_46447,46,Canary,46447,Gascon Malbec Mendoza,750mL,0,10.99,2024-12-31
115175,46_CANARY_46458,46,Canary,46458,Layer Cake Barosa Shiraz,750mL,0,15.99,2024-12-31
115176,46_CANARY_46476,46,Canary,46476,Tilia Malbec Mendoza,750mL,0,9.99,2024-12-31
115177,46_CANARY_46764,46,Canary,46764,Clayhouse Adobe Red Paso Rbl,750mL,0,11.99,2024-12-31


## 1-2. sales

### 1-2-1. EDA

In [18]:
sales.head()

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor
0,1_SPARROW_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_SPARROW_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_SPARROW_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_SPARROW_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_SPARROW_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [19]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12825363 entries, 0 to 12825362
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   inventory_id    object 
 1   store_id        int64  
 2   brand_id        int64  
 3   item            object 
 4   size            object 
 5   sales_quantity  int64  
 6   sales_dollars   float64
 7   sales_price     float64
 8   sales_date      object 
 9   item_volume     float64
 10  classification  int64  
 11  excise_tax      float64
 12  vendor_id       int64  
 13  vendor          object 
dtypes: float64(4), int64(5), object(5)
memory usage: 1.3+ GB


In [20]:
sales.nunique()

inventory_id      267552
store_id              80
brand_id           11237
item               10182
size                  51
sales_quantity       369
sales_dollars      10358
sales_price          420
sales_date           364
item_volume           30
classification         2
excise_tax          1187
vendor_id            127
vendor               130
dtype: int64

### 1-2-2. 날짜 관련 파생변수 칼럼 추가

In [134]:
sales['sales_date'] = pd.to_datetime(sales['sales_date'])

In [22]:
#월,요일 칼럼 추가
sales['month'] = sales['sales_date'].dt.month
sales['dayofweek'] = sales['sales_date'].dt.dayofweek # 월요일=0, 일요일=6

### 1-2-3. 결측치 처리

In [23]:
#결측치 확인
sales.isnull().sum()

inventory_id      0
store_id          0
brand_id          0
item              0
size              0
sales_quantity    0
sales_dollars     0
sales_price       0
sales_date        0
item_volume       0
classification    0
excise_tax        0
vendor_id         0
vendor            0
month             0
dayofweek         0
dtype: int64

In [24]:
if 'inventory_id' in sales.columns:
    id_parts = sales['inventory_id'].str.strip().str.split('_', expand=True)
    if id_parts.shape[1] >= 3:
        sales['store_id'] = id_parts[0].str.strip()
        sales['city'] = id_parts[1].str.strip()
        sales['brand_id'] = id_parts[2].str.strip()
        # 숫자 가능한 필드는 정수 변환 시도
        for col in ['store_id', 'brand_id']:
            sales[col] = pd.to_numeric(sales[col], errors='ignore')

In [25]:
sales['city'].unique()

array(['SPARROW', 'PIGEON', 'FINCH', 'ORIOLE', 'KINGFISHER',
       'MOCKINGBIRD', 'PELICAN', 'EAGLE', 'HAWK', 'OWL', 'FALCON',
       'ROBIN', 'VULTURE', 'CONDOR', 'DUCK', 'SWAN', 'GOOSE', 'HERON',
       'ALBATROSS', 'PENGUIN', 'STORK', 'CRANE', 'FLAMINGO', 'PUFFIN',
       'TERN', 'SANDPIPER', 'EGRET', 'SWALLOW', 'PARROT', 'CROW', 'MACAW',
       'COCKATOO', 'TOUCAN', 'HUMMINGBIRD', 'NIGHTINGALE', 'CANARY',
       'LOVEBIRD', 'WOODPECKER', 'TURKEY', 'SEAGULL', 'CHICKEN',
       'PHEASANT', 'GROUSE', 'PARTRIDGE', 'CUCKOO', 'DOVE', 'THRUSH',
       'FLICKER', 'JAY', 'CARDINAL', 'SHRIKE', 'OSTRICH', 'EMU', 'KIWI',
       'CASSOWARY', 'COOT', 'MAGPIE', 'LOON', 'HORNBILL', 'HOOPOE',
       'KITE', 'DODO', 'IBIS', 'GANNET', 'AUK', 'STARLING', 'BLUEBIRD',
       '', 'SPOONBILL'], dtype=object)

In [26]:
sales[sales['city'] == '']['store_id'].unique()

array([46, 81])

In [27]:
sales[sales['store_id']==46]

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor,month,dayofweek,city
411078,46_CANARY_10058,46,10058,F Coppola Dmd Ivry Cab Svgn,750mL,1,14.99,14.99,2024-01-16,750.0,2,0.11,2000,SOUTHERN WINE & SPIRITS NE,1,1,CANARY
411079,46_CANARY_10058,46,10058,F Coppola Dmd Ivry Cab Svgn,750mL,1,14.99,14.99,2024-01-31,750.0,2,0.11,2000,SOUTHERN WINE & SPIRITS NE,1,2,CANARY
411080,46_CANARY_10227,46,10227,Due Torri Pnt Nr del Venezie,750mL,3,29.97,9.99,2024-01-12,750.0,2,0.34,9165,ULTRA BEVERAGE COMPANY LLP,1,4,CANARY
411081,46_CANARY_10227,46,10227,Due Torri Pnt Nr del Venezie,750mL,2,19.98,9.99,2024-01-23,750.0,2,0.22,9165,ULTRA BEVERAGE COMPANY LLP,1,1,CANARY
411082,46_CANARY_1023,46,1023,Hennessy VSOP Cognac + 50mL,750mL,1,51.99,51.99,2024-01-25,750.0,1,0.79,8112,MOET HENNESSY USA INC,1,3,CANARY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6356128,46_CANARY_8166,46,8166,Mount Gay Eclipse,750mL,6,119.94,19.99,2024-07-27,750.0,1,4.72,7239,REMY COINTREAU USA INC,7,5,CANARY
6356129,46_CANARY_8288,46,8288,Korbel Brut,750mL,1,11.99,11.99,2024-07-07,750.0,2,0.11,1128,BROWN-FORMAN CORP,7,6,CANARY
6356130,46_CANARY_8323,46,8323,Jose Cuervo Gold w/Marg Mix,750mL,6,89.94,14.99,2024-07-27,750.0,1,4.72,7245,PROXIMO SPIRITS INC.,7,5,CANARY
6356131,46_CANARY_8339,46,8339,The Glenlivet 12 Yr Single,750mL,2,87.98,43.99,2024-07-07,750.0,1,1.57,17035,PERNOD RICARD USA,7,6,CANARY


In [28]:
sales[sales['store_id']==81]

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor,month,dayofweek,city
8169945,81__11321,81,11321,BV GDL Private Rsv Cab Svgn,750mL,3,359.97,119.99,2024-08-30,750.0,2,0.34,1590,DIAGEO CHATEAU ESTATE WINES,8,4,
8169946,81__11774,81,11774,Diamond Crk Cab Svgn Volcanc,750mL,2,449.98,224.99,2024-08-30,750.0,2,0.22,4425,MARTIGNETTI COMPANIES,8,4,
8169947,81__11775,81,11775,Diamond Creek Red Rock Cab S,750mL,2,449.98,224.99,2024-08-30,750.0,2,0.22,4425,MARTIGNETTI COMPANIES,8,4,
8169948,81__11776,81,11776,Diamond Crk Gravelly Cab Svg,750mL,3,674.97,224.99,2024-08-30,750.0,2,0.34,4425,MARTIGNETTI COMPANIES,8,4,
8169949,81__15971,81,15971,Revana Cab Svgn Napa Vly,750mL,3,329.97,109.99,2024-08-30,750.0,2,0.34,9165,ULTRA BEVERAGE COMPANY LLP,8,4,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12789939,81_SPOONBILL_984,81,984,Patron XO Cafe Liqueur,750mL,2,51.98,25.99,2024-12-22,750.0,1,1.57,9165,ULTRA BEVERAGE COMPANY LLP,12,6,SPOONBILL
12789940,81_SPOONBILL_984,81,984,Patron XO Cafe Liqueur,750mL,2,51.98,25.99,2024-12-23,750.0,1,1.57,9165,ULTRA BEVERAGE COMPANY LLP,12,0,SPOONBILL
12789941,81_SPOONBILL_984,81,984,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-24,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,1,SPOONBILL
12789942,81_SPOONBILL_984,81,984,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-26,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,3,SPOONBILL


#### ➡️ sales의 city 결측치를 store_id를 통해 {46: 'canary'}, {81:'spoonbill'}로 채우기

In [29]:
# 매핑 사전 정의
fill_map = {46: 'CANARY', 81: 'SPOONBILL'}

for store_id, city in fill_map.items():
    sales.loc[sales['store_id'] == store_id, 'city'] = city


In [30]:
for store_id, city in fill_map.items():
    mask = sales['store_id'] == store_id
    sales.loc[mask, 'city'] = sales.loc[mask, 'city'].fillna(city)


In [31]:
sales[sales["city"].isnull()]

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor,month,dayofweek,city


In [32]:
sales['inventory_id'] = sales['store_id'].astype(str) + '_' + sales['city'].astype(str).str.upper() + '_' + sales['brand_id'].astype(str)

### 1-2-4. size 칼럼 전처리

#### size 칼럼 문제: 단위가 혼재되어 있음

In [33]:
sales['size'].sort_values().unique()

array(['1.5L', '1.75L', '100mL', '100mL 4 Pk', '180mL', '187mL',
       '187mL 2 Pk', '187mL 3 Pk', '187mL 4 Pk', '18L', '19.5L', '200mL',
       '200mL 3 Pk', '200mL 4 Pk', '200mL 5 Pk', '20L', '22.0 Oz', '25',
       '25.0', '250mL', '250mL 4 Pk', '3/100mL', '300mL', '330mL',
       '375mL', '375mL 2 Pk', '375mL 3 Pk', '3L', '400mL', '4L', '5.0 Oz',
       '5/2 oz', '500mL', '50mL', '50mL 12 Pk', '50mL 3 Pk', '50mL 4 Pk',
       '50mL 5 Pk', '5L', '6L', '720mL', '750mL', '750mL + 1/',
       '750mL + 2/', '750mL + 3/', '750mL + 4/', '750mL 2 Pk',
       '750mL 3 Pk', '750mL 4 Pk', '9L', 'Liter'], dtype=object)

#### '2/' 같은 슬래시 패턴 포함된 데이터 확인

In [34]:
sales[sales['size'].str.contains('1/', na=False)]['item'].unique()

array(['Crown Royal Shot Glass VAP'], dtype=object)

In [35]:
sales[sales['size'].str.contains('2/', na=False)]['item'].unique()

array(['Glenmorangie Original VAP', 'Chivas Regal with2 50mLs'],
      dtype=object)

In [36]:
sales[sales['size'].str.contains(' 3/', na=False)]['item'].unique()

array(['Pinnacle Vodka Gift Pak', 'Three Olives Vodka w3 50mls'],
      dtype=object)

In [37]:
sales[sales['size'].str.contains('4/', na=False)]['item'].unique()

array(['Red Stag VAP 750+4/50ml'], dtype=object)

#### size 칼럼 전처리 -> 총 용량 나타내는 volume 칼럼 추가

In [38]:
import re
from fractions import Fraction

OZ_TO_ML = 29.5735       # 1 oz ≈ 29.5735 ml
GL_TO_ML = 3785.41     # 1 gallon ≈ 3785.41 ml
FREEBIE_UNIT_ML = 50.0   # 슬래시 뒤 증정 샷 1개의 기본 용량

def _safe_to_float(token: str) -> float | None:
    """정수·소수·분수 문자열을 안전하게 float로 변환."""
    token = token.strip().replace(',', '')
    if not token:
        return None
    try:
        return float(Fraction(token))
    except (ValueError, ZeroDivisionError):
        return None

def _collect_freebies(text: str, pattern: str, per_unit_ml: float = FREEBIE_UNIT_ML) -> tuple[str, float]:
    """증정품 패턴을 찾아 합산하고 본문에서 제거."""
    collected = 0.0
    def _repl(match):
        nonlocal collected
        count = int(match.group(1))
        collected += count * per_unit_ml
        return ' '
    return re.sub(pattern, _repl, text), collected

def _apply_pattern(text: str, pattern: str, converter) -> tuple[str, float]:
    """용량 패턴을 찾아 converter로 ml 환산, 누적 후 본문에서 제거."""
    collected = 0.0
    def _repl(match):
        nonlocal collected
        collected += converter(match)
        return ' '
    return re.sub(pattern, _repl, text), collected

def parse_size_to_total_volume(size: str, product_name: str | None = None) -> float:
    """size 문자열을 분석해 ml 단위 총 용량을 반환."""
    if pd.isna(size):
        return np.nan

    original = str(size).strip().lower()
    if not original:
        return np.nan

    # 25 ml 샷 15개 묶음(25, 25.0 등) 예외 처리 → 375 ml
    if re.fullmatch(r'25(?:\.0*)?', original):
        return 25.0 * 15

    s = original.replace('liter', '1l')
    s = re.sub(r'\s+', ' ', s)

    raw_has_50ml = '50ml' in s or '50 m' in s
    product_lower = str(product_name).lower() if product_name is not None else ''
    allow_slash_freebie = True
    if 'vap' in product_lower and not raw_has_50ml:
        allow_slash_freebie = False  # VAP인데 50ml 언급이 없으면 슬래시 보너스 무시

    total_ml = 0.0
    freebie_ml = 0.0

    def add_volume(value: str, unit: str, count: int = 1) -> float:
        """용량·단위를 ml로 변환 후 수량만큼 곱해 반환."""
        base = _safe_to_float(value)
        if base is None:
            return 0.0
        unit = unit.lower()
        if unit == 'ml':
            volume = base
        elif unit == 'l':
            volume = base * 1000.0
        elif unit == 'oz':
            volume = base * OZ_TO_ML
        elif unit == 'gal':   
            volume = base * GL_TO_ML 
        else:
            return 0.0
        return volume * count

    # '3/100ml' → 3 × 100ml 패턴 처리 (증정품으로 누적)
    def consume_fractional_packs(text: str) -> tuple[str, float]:
        collected = 0.0
        pattern = r'(\d+)\s*/\s*(\d+(?:\.\d+)?)(ml|l|oz|gal)'
        def _repl(match):
            nonlocal collected
            count = int(match.group(1))
            value = match.group(2)
            unit = match.group(3)
            collected += add_volume(value, unit, count)
            return ' '
        return re.sub(pattern, _repl, text), collected

    s, extra = consume_fractional_packs(s)
    freebie_ml += extra

    # 명시적 50ml 증정 패턴은 항상 반영
    for pattern in [
        r'(?:with|w)\s*(\d+)\s*50mls?',
        r'\+\s*(\d+)\s*/\s*50ml',
        r'\+\s*(\d+)\s*50mls?',
    ]:
        s, extra = _collect_freebies(s, pattern)
        freebie_ml += extra

    # “+ n/” 패턴은 조건부 반영
    slash_pattern = r'\+\s*(\d+)\s*/(?=\s|$)'
    if allow_slash_freebie:
        s, extra = _collect_freebies(s, slash_pattern)
        freebie_ml += extra
    else:
        s = re.sub(slash_pattern, ' ', s)

    # 본품 패턴을 순서대로 처리
    patterns = [
        (r'(?P<value>\d+(?:\.\d+)?(?:/\d+)?)\s*(?P<unit>ml|l|oz|gal)\s*(?P<count>\d+)\s*pk',
        lambda m: add_volume(m.group('value'), m.group('unit'), int(m.group('count')))),
        (r'(?P<value>\d+(?:\.\d+)?(?:/\d+)?)\s*(?P<unit>ml|l|oz|gal)x(?P<count>\d+)',
        lambda m: add_volume(m.group('value'), m.group('unit'), int(m.group('count')))),
        (r'(?P<count>\d+)\s*x\s*(?P<value>\d+(?:\.\d+)?(?:/\d+)?)\s*(?P<unit>ml|l|oz|gal)',
        lambda m: add_volume(m.group('value'), m.group('unit'), int(m.group('count')))),
        (r'(?P<value>\d+(?:\.\d+)?(?:/\d+)?)\s*(?P<unit>ml|l|oz|gal)',
        lambda m: add_volume(m.group('value'), m.group('unit'))),
    ]

    for pattern, converter in patterns:
        s, extra = _apply_pattern(s, pattern, converter)
        total_ml += extra

    total = total_ml + freebie_ml
    return total if total > 0 else np.nan


In [39]:
# item(상품명)을 함께 넘겨줌
sales['volume'] = sales.apply(
    lambda row: parse_size_to_total_volume(row['size'], row.get('item')),
    axis=1,
)

In [40]:
# 결과 확인
def summarize_size_conversion(df):
    summary = (
        df[['size', 'volume']]
        .groupby('size', dropna=False)
        .agg(
            rows=('volume', 'size'),  # 해당 size가 몇 건인지
            unique_volumes=('volume', lambda s: sorted({float(v) for v in s.dropna()})),
            sample_products=('volume', lambda s: s.head(3).tolist()),
        )
        .reset_index()
        .sort_values('rows', ascending=False)
    )
    return summary

display(summarize_size_conversion(sales))


,size,rows,unique_volumes,sample_products
41,750mL,7374318,[750.0],"[750.0, 750.0, 750.0]"
1,1.75L,2052760,[1750.0],"[1750.0, 1750.0, 1750.0]"
33,50mL,1262415,[50.0],"[50.0, 50.0, 50.0]"
0,1.5L,754235,[1500.0],"[1500.0, 1500.0, 1500.0]"
24,375mL,640170,[375.0],"[375.0, 375.0, 375.0]"
50,Liter,220366,[1000.0],"[1000.0, 1000.0, 1000.0]"
27,3L,159617,[3000.0],"[3000.0, 3000.0, 3000.0]"
38,5L,133659,[5000.0],"[5000.0, 5000.0, 5000.0]"
8,187mL 4 Pk,39330,[748.0],"[748.0, 748.0, 748.0]"
32,500mL,31667,[500.0],"[500.0, 500.0, 500.0]"


In [41]:
# 결과 확인
sales[sales['size']=='25.0'].head(3)

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor,month,dayofweek,city,volume
5819945,1_SPARROW_169,1,169,Twisted Shotz Multi Pack,25.0,3,50.97,16.99,2024-07-22,1100.0,1,3.46,9165,ULTRA BEVERAGE COMPANY LLP,7,0,SPARROW,375.0
5819946,1_SPARROW_169,1,169,Twisted Shotz Multi Pack,25.0,1,16.99,16.99,2024-07-23,1100.0,1,1.16,9165,ULTRA BEVERAGE COMPANY LLP,7,1,SPARROW,375.0
5819947,1_SPARROW_169,1,169,Twisted Shotz Multi Pack,25.0,1,16.99,16.99,2024-07-29,1100.0,1,1.16,9165,ULTRA BEVERAGE COMPANY LLP,7,0,SPARROW,375.0


##### ➡️ volume 칼럼이 잘 만들어진 것 확인

## 1-3. purchases

### 1-3-1. EDA

In [42]:
purchases.head()

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification
0,69_PENGUIN_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_HOOPOE_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1


In [43]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   inventory_id           object 
 1   store_id               int64  
 2   brand_id               int64  
 3   item                   object 
 4   size                   object 
 5   vendor_id              int64  
 6   vendor                 object 
 7   purchase_order_number  int64  
 8   purchase_order_date    object 
 9   receiving_date         object 
 10  invoice_date           object 
 11  pay_date               object 
 12  purchase_price         float64
 13  quantity               int64  
 14  dollars                float64
 15  classification         int64  
dtypes: float64(2), int64(6), object(8)
memory usage: 289.6+ MB


In [44]:
purchases.isnull().sum()

inventory_id             0
store_id                 0
brand_id                 0
item                     0
size                     3
vendor_id                0
vendor                   0
purchase_order_number    0
purchase_order_date      0
receiving_date           0
invoice_date             0
pay_date                 0
purchase_price           0
quantity                 0
dollars                  0
classification           0
dtype: int64

### 1-3-2. 결측치 처리

In [45]:
if 'inventory_id' in purchases.columns:
    id_parts = purchases['inventory_id'].str.strip().str.split('_', expand=True)
    if id_parts.shape[1] >= 3:
        purchases['store_id'] = id_parts[0].str.strip()
        purchases['city'] = id_parts[1].str.strip()
        purchases['brand_id'] = id_parts[2].str.strip()
        # 숫자 가능한 필드는 정수 변환 시도
        for col in ['store_id', 'brand_id']:
            sales[col] = pd.to_numeric(purchases[col], errors='ignore')

In [46]:
purchases['city'].unique()

array(['PENGUIN', 'FLAMINGO', 'PUFFIN', 'SPARROW', 'HOOPOE', 'SEAGULL',
       'GANNET', 'DUCK', 'SWALLOW', 'CROW', 'PELICAN', 'MAGPIE', 'OWL',
       'LOVEBIRD', 'PARTRIDGE', 'PHEASANT', 'SWAN', 'EGRET', 'EAGLE',
       'AUK', 'CARDINAL', 'CRANE', 'MACAW', 'PIGEON', 'COOT',
       'WOODPECKER', 'KIWI', 'COCKATOO', 'BLUEBIRD', 'ALBATROSS',
       'SANDPIPER', 'GROUSE', 'KITE', 'TOUCAN', 'MOCKINGBIRD', 'CANARY',
       'IBIS', 'VULTURE', 'HORNBILL', 'CHICKEN', 'HUMMINGBIRD', 'DODO',
       'HERON', 'CUCKOO', 'TURKEY', 'SHRIKE', 'ORIOLE', 'KINGFISHER',
       'STORK', 'DOVE', 'EMU', 'JAY', 'GOOSE', 'STARLING', 'OSTRICH',
       'FALCON', 'HAWK', 'CONDOR', 'FLICKER', 'NIGHTINGALE', 'THRUSH',
       'FINCH', 'PARROT', 'ROBIN', 'TERN', 'LOON', 'CASSOWARY', '',
       'SPOONBILL'], dtype=object)

In [47]:
purchases[purchases['city'] == '']['store_id'].unique()

array(['46', '81'], dtype=object)

In [48]:
# 매핑 사전 정의
fill_map = {46: 'CANARY', 81: 'SPOONBILL'}

for store_id, city in fill_map.items():
    purchases.loc[purchases['store_id'] == store_id, 'city'] = city

for store_id, city in fill_map.items():
    mask = purchases['store_id'] == store_id
    purchases.loc[mask, 'city'] = purchases.loc[mask, 'city'].fillna(city)


In [49]:
nan_rows = purchases[purchases.isna().any(axis=1)]
nan_rows

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,city
1109668,34_PUFFIN_3121,34,3121,Pinnacle Rainbow Sherbet,NaN,12546,JIM BEAM BRANDS COMPANY,10938,2024-06-27,2024-07-04,2024-07-13,2024-08-16,6.93,7,48.51,1,PUFFIN
1112426,34_PUFFIN_5678,34,5678,Skinnygirl Pina Colada,NaN,12546,JIM BEAM BRANDS COMPANY,10938,2024-06-27,2024-07-09,2024-07-13,2024-08-16,6.93,6,41.58,1,PUFFIN
1116302,39_PARROT_15365,39,15365,Alabaster 07 Tinta de Toro,NaN,9552,M S WALKER INC,10972,2024-06-29,2024-07-07,2024-07-13,2024-08-21,91.83,1,91.83,2,PARROT


In [50]:
# 사이즈의 Nan 값을 동일 item의 다른 구매내역을 이용하여 대체하기
#Pinnacle Rainbow Sherbet의 사이즈는 750ml로 대체
#Skinnygirl Pina Colada의 사이즈도 750ml로 대체
#Alabaster 07 Tinta de Toro’는 다른 구매내역은 없지만 sales 데이터와 inventory 데이터에서 찾아보니 모두 750ml이므로 750ml로 결측치를 대체
purchases.loc[purchases['item']=='Alabaster 07 Tinta de Toro']

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,city
1116302,39_PARROT_15365,39,15365,Alabaster 07 Tinta de Toro,NaN,9552,M S WALKER INC,10972,2024-06-29,2024-07-07,2024-07-13,2024-08-21,91.83,1,91.83,2,PARROT


In [51]:
# 결측치 대체
purchases['size'] = purchases['size'].fillna('750mL')
purchases.isnull().sum()


inventory_id             0
store_id                 0
brand_id                 0
item                     0
size                     0
vendor_id                0
vendor                   0
purchase_order_number    0
purchase_order_date      0
receiving_date           0
invoice_date             0
pay_date                 0
purchase_price           0
quantity                 0
dollars                  0
classification           0
city                     0
dtype: int64

## 1-4. purchase_prices

### 1-4-1. EDA

In [52]:
purchase_prices.head()

,brand_id,item,price,size,item_volume,classification,purchase_price,vendor_id,vendor
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [53]:
purchase_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12261 entries, 0 to 12260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand_id        12261 non-null  int64  
 1   item            12260 non-null  object 
 2   price           12261 non-null  float64
 3   size            12260 non-null  object 
 4   item_volume     12260 non-null  object 
 5   classification  12261 non-null  int64  
 6   purchase_price  12261 non-null  float64
 7   vendor_id       12261 non-null  int64  
 8   vendor          12261 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 862.2+ KB


In [54]:
purchase_prices.isnull().sum()

brand_id          0
item              1
price             0
size              1
item_volume       1
classification    0
purchase_price    0
vendor_id         0
vendor            0
dtype: int64

### 1-4-2. 결측치 처리

In [55]:
# price가 0인 값이 2개 있음! 놀랍게도 이는 결측치가 있는 행과 일치함!
(purchase_prices['price'] == 0).sum()

2

In [56]:
nan_rows2 = purchase_prices[purchase_prices.isna().any(axis=1)]
nan_rows2

,brand_id,item,price,size,item_volume,classification,purchase_price,vendor_id,vendor
7915,4202,NaN,0.0,NaN,NaN,1,11.19,480,BACARDI USA INC


In [57]:
# 1. brand_id = 4202 인 행의 vendor인 'BACARDI USA INC'를 purchases 데이터에서 찾아본 결과,
# 이 vendor에서는 Bacardi Grapefruit Rum이라는 item 한 가지만 매입함(brand_id=4092)
# 따라서 brand_id = 4202인 행은 잘못된 정보로 간주하고 삭제하기로 결정함.

# 2. brand_id = 2166 인 행은 price가 0임.
# sales 데이터에서 찾아보니 '54.99->785번 / 59.99->153번 / 65.99->124번'으로 나타남. 똑같은 아이템, 사이즈 이기 때문에 무엇으로 해야할지는 모르겠음!
# price가 0인 상품이 있다면 그것은 sales할 때 여러가지 가격으로 판매했기 때문임!
# 일단 이 행의 price는 최빈값인 54.99로 대체하기로 결정함.

purchases.loc[(purchases['vendor_id']==480) & (purchases['purchase_price']==11.19)].head()

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,city
936582,79_AUK_4092,79,4092,Bacardi Grapefruit Rum,750mL,480,BACARDI USA INC,10550,2024-06-06,2024-06-18,2024-06-27,2024-08-05,11.19,11,123.09,1,AUK
936661,54_PARTRIDGE_4092,54,4092,Bacardi Grapefruit Rum,750mL,480,BACARDI USA INC,10550,2024-06-06,2024-06-17,2024-06-27,2024-08-05,11.19,12,134.28,1,PARTRIDGE
936693,61_SHRIKE_4092,61,4092,Bacardi Grapefruit Rum,750mL,480,BACARDI USA INC,10550,2024-06-06,2024-06-17,2024-06-27,2024-08-05,11.19,12,134.28,1,SHRIKE
937196,7_MAGPIE_4092,7,4092,Bacardi Grapefruit Rum,750mL,480,BACARDI USA INC,10550,2024-06-06,2024-06-17,2024-06-27,2024-08-05,11.19,12,134.28,1,MAGPIE
937356,41_STORK_4092,41,4092,Bacardi Grapefruit Rum,750mL,480,BACARDI USA INC,10550,2024-06-06,2024-06-17,2024-06-27,2024-08-05,11.19,12,134.28,1,STORK


In [58]:
# brand_id=4202인 행 삭제
purchase_prices = purchase_prices[purchase_prices['brand_id'] != 4202].copy()
# brand_id=2166인 행의 price를 최빈값인 54.99로 대체
purchase_prices.loc[(purchase_prices['brand_id'] == 2166) & (purchase_prices['price'] == 0.00), 'price'] = 54.99

purchase_prices.head()

,brand_id,item,price,size,item_volume,classification,purchase_price,vendor_id,vendor
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


## 1-5. vendor_invoice

### 1-5-1. EDA

In [59]:
vendor_invoice.head()

,vendor_id,vendor,invoice_date,purchase_order_number,purchase_order_date,pay_date,quantity,dollars,freight,approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,NaN
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,NaN
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,NaN
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,NaN
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,NaN


In [60]:
vendor_invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   vendor_id              5543 non-null   int64  
 1   vendor                 5543 non-null   object 
 2   invoice_date           5543 non-null   object 
 3   purchase_order_number  5543 non-null   int64  
 4   purchase_order_date    5543 non-null   object 
 5   pay_date               5543 non-null   object 
 6   quantity               5543 non-null   int64  
 7   dollars                5543 non-null   float64
 8   freight                5543 non-null   float64
 9   approval               374 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 433.2+ KB


In [61]:
vendor_invoice.isnull().sum()

vendor_id                   0
vendor                      0
invoice_date                0
purchase_order_number       0
purchase_order_date         0
pay_date                    0
quantity                    0
dollars                     0
freight                     0
approval                 5169
dtype: int64

### 1-5-2. 결측치 처리

#### approval의 의미 탐색

In [62]:
#approval이 Nan이 아닌 행만 추출. 즉 허가된 것만 추출.
ven_apr=vendor_invoice.dropna(subset=['approval'])
ven_apr

,vendor_id,vendor,invoice_date,purchase_order_number,purchase_order_date,pay_date,quantity,dollars,freight,approval
93,480,BACARDI USA INC,2024-01-19,8288,2024-01-01,2024-02-18,32463,396606.13,1864.05,Frank Delahunt
97,1128,BROWN-FORMAN CORP,2024-01-15,8210,2023-12-27,2024-02-25,23351,298902.89,1494.51,Frank Delahunt
109,3960,DIAGEO NORTH AMERICA INC,2024-01-14,8271,2023-12-31,2024-02-11,100183,954454.16,4581.38,Frank Delahunt
126,12546,JIM BEAM BRANDS COMPANY,2024-01-16,8241,2023-12-29,2024-02-15,56213,559691.95,2910.40,Frank Delahunt
137,4425,MARTIGNETTI COMPANIES,2024-01-18,8276,2023-12-31,2024-02-28,42400,458222.53,2336.93,Frank Delahunt
...,...,...,...,...,...,...,...,...,...,...
5487,4425,MARTIGNETTI COMPANIES,2025-01-03,13584,2024-12-19,2025-02-17,92470,1010674.21,4952.30,Frank Delahunt
5490,8112,MOET HENNESSY USA INC,2025-01-03,13635,2024-12-22,2025-02-12,15003,316863.94,1457.57,Frank Delahunt
5497,17035,PERNOD RICARD USA,2025-01-03,13597,2024-12-20,2025-02-16,48425,663584.35,3649.71,Frank Delahunt
5507,8004,SAZERAC CO INC,2025-01-10,13567,2024-12-18,2025-02-19,43240,318075.65,1494.96,Frank Delahunt


In [63]:
#허가는 다 Frank라고 되어있는 것을 확인함
ven_apr['approval'].value_counts()

Frank Delahunt    374
Name: approval, dtype: int64

In [64]:
# 허가(1)와 비허가(0)를 정수값으로 바꿈. 
vendor_invoice['approval'] = vendor_invoice['approval'].fillna(0)
vendor_invoice=vendor_invoice.replace('Frank Delahunt', 1)
vendor_invoice['approval'].value_counts()

0    5169
1     374
Name: approval, dtype: int64

In [65]:
vendor_invoice.isnull().sum()

vendor_id                0
vendor                   0
invoice_date             0
purchase_order_number    0
purchase_order_date      0
pay_date                 0
quantity                 0
dollars                  0
freight                  0
approval                 0
dtype: int64

# 2. 데이터 병합

## 2-1. 재고 데이터: inventory

- begin_inventory
- end_inventory
- purchases_price

### 2-1-1. 칼럼명 변경

In [66]:
begin_inventory.columns

Index(['inventory_id', 'store_id', 'city', 'brand_id', 'item', 'size',
       'on_hand', 'price', 'begin_date'],
      dtype='object')

In [67]:
# 칼럼명 변경
begin_inventory.columns = ['inventory_id', 'store_id', 'city', 'brand_id', 'item', 'size', 'on_hand_begin', 'sales_price_begin', 'begin_date']
begin_inventory.head()

,inventory_id,store_id,city,brand_id,item,size,on_hand_begin,sales_price_begin,begin_date
0,1_SPARROW_58,1,Sparrow,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_SPARROW_60,1,Sparrow,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_SPARROW_62,1,Sparrow,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_SPARROW_63,1,Sparrow,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_SPARROW_72,1,Sparrow,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


In [68]:
end_inventory.columns

Index(['inventory_id', 'store_id', 'city', 'brand_id', 'item', 'size',
       'on_hand', 'price', 'end_date'],
      dtype='object')

In [69]:
end_inventory.columns = ['inventory_id', 'store_id', 'city', 'brand_id', 'item', 'size', 'on_hand_end', 'sales_price_end', 'end_date']
end_inventory.head()

,inventory_id,store_id,city,brand_id,item,size,on_hand_end,sales_price_end,end_date
0,1_SPARROW_58,1,Sparrow,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_SPARROW_62,1,Sparrow,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_SPARROW_63,1,Sparrow,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_SPARROW_72,1,Sparrow,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_SPARROW_75,1,Sparrow,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


### 2-1-2. 재고 데이터 병합 (기초 + 기말)

In [72]:
inventory = pd.merge(begin_inventory, end_inventory, on=['inventory_id', 'store_id', 'city', 'brand_id', 'item', 'size'], how='outer')
inventory

,inventory_id,store_id,city,brand_id,item,size,on_hand_begin,sales_price_begin,begin_date,on_hand_end,sales_price_end,end_date
0,1_SPARROW_58,1,Sparrow,58,Gekkeikan Black & Gold Sake,750mL,8.0,12.99,2024-01-01,11.0,12.99,2024-12-31
1,1_SPARROW_60,1,Sparrow,60,Canadian Club 1858 VAP,750mL,7.0,10.99,2024-01-01,NaN,NaN,NaN
2,1_SPARROW_62,1,Sparrow,62,Herradura Silver Tequila,750mL,6.0,36.99,2024-01-01,7.0,36.99,2024-12-31
3,1_SPARROW_63,1,Sparrow,63,Herradura Reposado Tequila,750mL,3.0,38.99,2024-01-01,7.0,38.99,2024-12-31
4,1_SPARROW_72,1,Sparrow,72,No. 3 London Dry Gin,750mL,6.0,34.99,2024-01-01,4.0,34.99,2024-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
257491,81_SPOONBILL_90087,81,Spoonbill,90087,Ch Mouton Rothschild 12 Paui,750mL,NaN,NaN,NaN,3.0,469.99,2024-12-31
257492,81_SPOONBILL_90088,81,Spoonbill,90088,Ch Le Petite Mouton 12 Pauil,750mL,NaN,NaN,NaN,3.0,134.99,2024-12-31
257493,81_SPOONBILL_90089,81,Spoonbill,90089,Ch Lynch Bages 12 Pauillac,750mL,NaN,NaN,NaN,3.0,119.99,2024-12-31
257494,81_SPOONBILL_90090,81,Spoonbill,90090,Ch Lafite Rothschild 12,750mL,NaN,NaN,NaN,3.0,649.99,2024-12-31


In [73]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257496 entries, 0 to 257495
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   inventory_id       257496 non-null  object 
 1   store_id           257496 non-null  int64  
 2   city               257496 non-null  object 
 3   brand_id           257496 non-null  int64  
 4   item               257496 non-null  object 
 5   size               257496 non-null  object 
 6   on_hand_begin      206529 non-null  float64
 7   sales_price_begin  206529 non-null  float64
 8   begin_date         206529 non-null  object 
 9   on_hand_end        224489 non-null  float64
 10  sales_price_end    224489 non-null  float64
 11  end_date           224489 non-null  object 
dtypes: float64(4), int64(2), object(6)
memory usage: 25.5+ MB


In [74]:
inventory.isnull().sum()

inventory_id             0
store_id                 0
city                     0
brand_id                 0
item                     0
size                     0
on_hand_begin        50967
sales_price_begin    50967
begin_date           50967
on_hand_end          33007
sales_price_end      33007
end_date             33007
dtype: int64

### 2-1-3. (기말 - 기초) 재고 차이 파생변수 칼럼 추가 (on_hand_diff)

In [75]:
inventory['on_hand_diff'] = inventory['on_hand_end'] - inventory['on_hand_begin']
inventory

,inventory_id,store_id,city,brand_id,item,size,on_hand_begin,sales_price_begin,begin_date,on_hand_end,sales_price_end,end_date,on_hand_diff
0,1_SPARROW_58,1,Sparrow,58,Gekkeikan Black & Gold Sake,750mL,8.0,12.99,2024-01-01,11.0,12.99,2024-12-31,3.0
1,1_SPARROW_60,1,Sparrow,60,Canadian Club 1858 VAP,750mL,7.0,10.99,2024-01-01,NaN,NaN,NaN,NaN
2,1_SPARROW_62,1,Sparrow,62,Herradura Silver Tequila,750mL,6.0,36.99,2024-01-01,7.0,36.99,2024-12-31,1.0
3,1_SPARROW_63,1,Sparrow,63,Herradura Reposado Tequila,750mL,3.0,38.99,2024-01-01,7.0,38.99,2024-12-31,4.0
4,1_SPARROW_72,1,Sparrow,72,No. 3 London Dry Gin,750mL,6.0,34.99,2024-01-01,4.0,34.99,2024-12-31,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257491,81_SPOONBILL_90087,81,Spoonbill,90087,Ch Mouton Rothschild 12 Paui,750mL,NaN,NaN,NaN,3.0,469.99,2024-12-31,NaN
257492,81_SPOONBILL_90088,81,Spoonbill,90088,Ch Le Petite Mouton 12 Pauil,750mL,NaN,NaN,NaN,3.0,134.99,2024-12-31,NaN
257493,81_SPOONBILL_90089,81,Spoonbill,90089,Ch Lynch Bages 12 Pauillac,750mL,NaN,NaN,NaN,3.0,119.99,2024-12-31,NaN
257494,81_SPOONBILL_90090,81,Spoonbill,90090,Ch Lafite Rothschild 12,750mL,NaN,NaN,NaN,3.0,649.99,2024-12-31,NaN


In [76]:
inventory.isnull().sum()

inventory_id             0
store_id                 0
city                     0
brand_id                 0
item                     0
size                     0
on_hand_begin        50967
sales_price_begin    50967
begin_date           50967
on_hand_end          33007
sales_price_end      33007
end_date             33007
on_hand_diff         83974
dtype: int64

In [135]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257496 entries, 0 to 257495
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   inventory_id       257496 non-null  object 
 1   store_id           257496 non-null  int64  
 2   city               257496 non-null  object 
 3   brand_id           257496 non-null  int64  
 4   item               257496 non-null  object 
 5   size               257496 non-null  object 
 6   on_hand_begin      206529 non-null  float64
 7   sales_price_begin  206529 non-null  float64
 8   begin_date         206529 non-null  object 
 9   on_hand_end        224489 non-null  float64
 10  sales_price_end    224489 non-null  float64
 11  end_date           224489 non-null  object 
 12  on_hand_diff       173522 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 27.5+ MB


## 2-2. 공급 데이터: purchase_vendor

- purchases
- vendor_invoice
- purchase_prices

In [77]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   inventory_id           object 
 1   store_id               object 
 2   brand_id               object 
 3   item                   object 
 4   size                   object 
 5   vendor_id              int64  
 6   vendor                 object 
 7   purchase_order_number  int64  
 8   purchase_order_date    object 
 9   receiving_date         object 
 10  invoice_date           object 
 11  pay_date               object 
 12  purchase_price         float64
 13  quantity               int64  
 14  dollars                float64
 15  classification         int64  
 16  city                   object 
dtypes: float64(2), int64(4), object(11)
memory usage: 307.7+ MB


In [78]:
vendor_invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   vendor_id              5543 non-null   int64  
 1   vendor                 5543 non-null   object 
 2   invoice_date           5543 non-null   object 
 3   purchase_order_number  5543 non-null   int64  
 4   purchase_order_date    5543 non-null   object 
 5   pay_date               5543 non-null   object 
 6   quantity               5543 non-null   int64  
 7   dollars                5543 non-null   float64
 8   freight                5543 non-null   float64
 9   approval               5543 non-null   int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 433.2+ KB


### 2-2-1. 1차 병합 시도

In [79]:
purchases_vendor = pd.merge(purchases, vendor_invoice, how='left')

In [80]:
purchases_vendor

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,city,freight,approval
0,69_PENGUIN_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1,PENGUIN,3.47,0.0
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1,FLAMINGO,NaN,NaN
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1,PUFFIN,NaN,NaN
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1,SPARROW,NaN,NaN
4,76_HOOPOE_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1,HOOPOE,4.61,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_TURKEY_22298,49,22298,Zorvino Vyds Sangiovese,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-28,2025-01-09,2025-02-06,8.06,12,96.72,2,TURKEY,NaN,NaN
2372470,1_SPARROW_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-27,2025-01-09,2025-02-06,9.39,12,112.68,2,SPARROW,NaN,NaN
2372471,66_CROW_22297,66,22297,Zorvino Vyds Pearz,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,2025-01-09,2025-02-06,6.75,12,81.00,2,CROW,NaN,NaN
2372472,69_PENGUIN_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,2025-01-09,2025-02-06,9.39,12,112.68,2,PENGUIN,NaN,NaN


##### ➡️ freight, approval이 NaN인 행이 생겨버림

In [81]:
vendor_invoice[vendor_invoice['purchase_order_number']==8137]

,vendor_id,vendor,invoice_date,purchase_order_number,purchase_order_date,pay_date,quantity,dollars,freight,approval
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,0


In [82]:
purchases[purchases['purchase_order_number']==8137]

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,city
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1,FLAMINGO
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1,PUFFIN
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1,SPARROW


##### ➡️ 하나의 송장(purchase_order_number)인데 여러 지점들에게 발주하는 형태인 경우, quantity 칼럼에 차이가 생겨서 NaN으로 찍힌 것임.

### 2-2-2. volume 칼럼 추가 후 부피 기준 운송비 배분하여 병합

In [83]:
# item(상품명)을 함께 넘겨줌
purchases['volume'] = purchases.apply(
    lambda row: parse_size_to_total_volume(row['size'], row.get('item')),
    axis=1,
)

In [84]:
# 1️⃣ 각 행의 실제 부피 계산 (개별 상품 부피 × 수량)
purchases['total_volume'] = purchases['volume'] * purchases['quantity']

# 2️⃣ 주문번호별 총 부피 계산
order_volume = (
    purchases.groupby('purchase_order_number')['total_volume']
    .sum()
    .rename('order_volume')
)

In [85]:
# 3️⃣ 병합
purchases_vendor = purchases.merge(order_volume, on='purchase_order_number', how='left')
purchases_vendor

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,invoice_date,pay_date,purchase_price,quantity,dollars,classification,city,volume,total_volume,order_volume
0,69_PENGUIN_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1,PENGUIN,750.0,4500.0,4500.0
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1,FLAMINGO,1750.0,7000.0,26250.0
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1,PUFFIN,1750.0,8750.0,26250.0
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1,SPARROW,1750.0,10500.0,26250.0
4,76_HOOPOE_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1,HOOPOE,750.0,3750.0,3750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_TURKEY_22298,49,22298,Zorvino Vyds Sangiovese,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-28,2025-01-09,2025-02-06,8.06,12,96.72,2,TURKEY,750.0,9000.0,327750.0
2372470,1_SPARROW_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-27,2025-01-09,2025-02-06,9.39,12,112.68,2,SPARROW,750.0,9000.0,327750.0
2372471,66_CROW_22297,66,22297,Zorvino Vyds Pearz,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,2025-01-09,2025-02-06,6.75,12,81.00,2,CROW,750.0,9000.0,327750.0
2372472,69_PENGUIN_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,2025-01-09,2025-02-06,9.39,12,112.68,2,PENGUIN,750.0,9000.0,327750.0


In [86]:
# 4️⃣ 주문번호별 freight (운송비)
freight_per_order = vendor_invoice[['purchase_order_number', 'freight']].drop_duplicates()

# 5️⃣ freight 확장
purchases_vendor = purchases_vendor.merge(freight_per_order, on='purchase_order_number', how='left')

# 6️⃣ 운송비 배분 (부피×수량 기준)
purchases_vendor['freight_alloc'] = (
    purchases_vendor['freight'] *
    (purchases_vendor['total_volume'] / purchases_vendor['order_volume'])
)
purchases_vendor

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,...,purchase_price,quantity,dollars,classification,city,volume,total_volume,order_volume,freight,freight_alloc
0,69_PENGUIN_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,...,35.71,6,214.26,1,PENGUIN,750.0,4500.0,4500.0,3.47,3.470000
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,...,9.35,4,37.40,1,FLAMINGO,1750.0,7000.0,26250.0,8.57,2.285333
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,...,9.41,5,47.05,1,PUFFIN,1750.0,8750.0,26250.0,8.57,2.856667
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,...,9.35,6,56.10,1,SPARROW,1750.0,10500.0,26250.0,8.57,3.428000
4,76_HOOPOE_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,...,21.32,5,106.60,1,HOOPOE,750.0,3750.0,3750.0,4.61,4.610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_TURKEY_22298,49,22298,Zorvino Vyds Sangiovese,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-28,...,8.06,12,96.72,2,TURKEY,750.0,9000.0,327750.0,4.16,0.114233
2372470,1_SPARROW_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-27,...,9.39,12,112.68,2,SPARROW,750.0,9000.0,327750.0,4.16,0.114233
2372471,66_CROW_22297,66,22297,Zorvino Vyds Pearz,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,...,6.75,12,81.00,2,CROW,750.0,9000.0,327750.0,4.16,0.114233
2372472,69_PENGUIN_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,...,9.39,12,112.68,2,PENGUIN,750.0,9000.0,327750.0,4.16,0.114233


In [87]:
# 7️⃣ 운송비 포함 매입단가 계산
purchases_vendor['purchase_cost'] = purchases_vendor['purchase_price'] + (purchases_vendor['freight_alloc']/ purchases_vendor['quantity'])
purchases_vendor.head()

,inventory_id,store_id,brand_id,item,size,vendor_id,vendor,purchase_order_number,purchase_order_date,receiving_date,...,quantity,dollars,classification,city,volume,total_volume,order_volume,freight,freight_alloc,purchase_cost
0,69_PENGUIN_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,...,6,214.26,1,PENGUIN,750.0,4500.0,4500.0,3.47,3.470000,36.288333
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,...,4,37.40,1,FLAMINGO,1750.0,7000.0,26250.0,8.57,2.285333,9.921333
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,...,5,47.05,1,PUFFIN,1750.0,8750.0,26250.0,8.57,2.856667,9.981333
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,...,6,56.10,1,SPARROW,1750.0,10500.0,26250.0,8.57,3.428000,9.921333
4,76_HOOPOE_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,...,5,106.60,1,HOOPOE,750.0,3750.0,3750.0,4.61,4.610000,22.242000


In [88]:
purchases_vendor.columns

Index(['inventory_id', 'store_id', 'brand_id', 'item', 'size', 'vendor_id',
       'vendor', 'purchase_order_number', 'purchase_order_date',
       'receiving_date', 'invoice_date', 'pay_date', 'purchase_price',
       'quantity', 'dollars', 'classification', 'city', 'volume',
       'total_volume', 'order_volume', 'freight', 'freight_alloc',
       'purchase_cost'],
      dtype='object')

In [89]:
# 칼럼 순서 변경
purchases_vendor = purchases_vendor[['inventory_id', 'store_id', 'brand_id', 'item', 'size', 'volume', 'total_volume', 'order_volume',
                                     'classification',
                                     'vendor_id', 'vendor',
                                     'purchase_order_number', 'purchase_order_date', 'receiving_date', 'invoice_date', 'pay_date',
                                     'dollars', 'quantity', 'purchase_price', 'purchase_cost',
                                     'freight', 'freight_alloc']]
purchases_vendor

,inventory_id,store_id,brand_id,item,size,volume,total_volume,order_volume,classification,vendor_id,...,purchase_order_date,receiving_date,invoice_date,pay_date,dollars,quantity,purchase_price,purchase_cost,freight,freight_alloc
0,69_PENGUIN_8412,69,8412,Tequila Ocho Plata Fresno,750mL,750.0,4500.0,4500.0,1,105,...,2023-12-21,2024-01-02,2024-01-04,2024-02-16,214.26,6,35.71,36.288333,3.47,3.470000
1,30_FLAMINGO_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,1750.0,7000.0,26250.0,1,4466,...,2023-12-22,2024-01-01,2024-01-07,2024-02-21,37.40,4,9.35,9.921333,8.57,2.285333
2,34_PUFFIN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,1750.0,8750.0,26250.0,1,4466,...,2023-12-22,2024-01-02,2024-01-07,2024-02-21,47.05,5,9.41,9.981333,8.57,2.856667
3,1_SPARROW_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,1750.0,10500.0,26250.0,1,4466,...,2023-12-22,2024-01-01,2024-01-07,2024-02-21,56.10,6,9.35,9.921333,8.57,3.428000
4,76_HOOPOE_2034,76,2034,Glendalough Double Barrel,750mL,750.0,3750.0,3750.0,1,388,...,2023-12-24,2024-01-02,2024-01-09,2024-02-16,106.60,5,21.32,22.242000,4.61,4.610000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_TURKEY_22298,49,22298,Zorvino Vyds Sangiovese,750mL,750.0,9000.0,327750.0,2,90058,...,2024-12-19,2024-12-28,2025-01-09,2025-02-06,96.72,12,8.06,8.069519,4.16,0.114233
2372470,1_SPARROW_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,750.0,9000.0,327750.0,2,90058,...,2024-12-19,2024-12-27,2025-01-09,2025-02-06,112.68,12,9.39,9.399519,4.16,0.114233
2372471,66_CROW_22297,66,22297,Zorvino Vyds Pearz,750mL,750.0,9000.0,327750.0,2,90058,...,2024-12-19,2024-12-26,2025-01-09,2025-02-06,81.00,12,6.75,6.759519,4.16,0.114233
2372472,69_PENGUIN_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,750.0,9000.0,327750.0,2,90058,...,2024-12-19,2024-12-26,2025-01-09,2025-02-06,112.68,12,9.39,9.399519,4.16,0.114233


In [90]:
purchases_vendor.isnull().sum()

inventory_id             0
store_id                 0
brand_id                 0
item                     0
size                     0
volume                   0
total_volume             0
order_volume             0
classification           0
vendor_id                0
vendor                   0
purchase_order_number    0
purchase_order_date      0
receiving_date           0
invoice_date             0
pay_date                 0
dollars                  0
quantity                 0
purchase_price           0
purchase_cost            0
freight                  0
freight_alloc            0
dtype: int64

### 2-2-3. 리드 타임(lead time) 파생변수 칼럼 추가

In [91]:
# 날짜형 변환
purchases_vendor['purchase_order_date'] = pd.to_datetime(purchases_vendor['purchase_order_date'])
purchases_vendor['receiving_date'] = pd.to_datetime(purchases_vendor['receiving_date'])

# 리드타임 계산 (일 단위)
purchases_vendor['lead_time'] = (purchases_vendor['receiving_date'] - purchases_vendor['purchase_order_date']).dt.days

# 확인
purchases_vendor[['purchase_order_date', 'receiving_date', 'lead_time']].head()


/var/folders/br/f3bzbxv50x36ysy4gwqml73h0000gn/T/ipykernel_73096/3907745753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_vendor['purchase_order_date'] = pd.to_datetime(purchases_vendor['purchase_order_date'])
/var/folders/br/f3bzbxv50x36ysy4gwqml73h0000gn/T/ipykernel_73096/3907745753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_vendor['receiving_date'] = pd.to_datetime(purchases_vendor['receiving_date'])


,purchase_order_date,receiving_date,lead_time
0,2023-12-21,2024-01-02,12
1,2023-12-22,2024-01-01,10
2,2023-12-22,2024-01-02,11
3,2023-12-22,2024-01-01,10
4,2023-12-24,2024-01-02,9


In [92]:
purchases_vendor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2372474 entries, 0 to 2372473
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   inventory_id           object        
 1   store_id               object        
 2   brand_id               object        
 3   item                   object        
 4   size                   object        
 5   volume                 float64       
 6   total_volume           float64       
 7   order_volume           float64       
 8   classification         int64         
 9   vendor_id              int64         
 10  vendor                 object        
 11  purchase_order_number  int64         
 12  purchase_order_date    datetime64[ns]
 13  receiving_date         datetime64[ns]
 14  invoice_date           object        
 15  pay_date               object        
 16  dollars                float64       
 17  quantity               int64         
 18  purchase_price        

## 2-3. 판매 데이터: sales

### 2-3-1. EDA

In [93]:
sales

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor,month,dayofweek,city,volume
0,1_SPARROW_1004,69.0,8412.0,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,0,SPARROW,750.0
1,1_SPARROW_1004,30.0,5255.0,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY,1,1,SPARROW,750.0
2,1_SPARROW_1004,34.0,5215.0,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,2,SPARROW,750.0
3,1_SPARROW_1004,1.0,5255.0,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,0,SPARROW,750.0
4,1_SPARROW_1005,76.0,2034.0,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,1,SPARROW,750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12825358,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-17,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,1,BLUEBIRD,750.0
12825359,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-21,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,5,BLUEBIRD,750.0
12825360,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,3,77.97,25.99,2024-12-23,750.0,1,2.36,9165,ULTRA BEVERAGE COMPANY LLP,12,0,BLUEBIRD,750.0
12825361,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-24,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,1,BLUEBIRD,750.0


In [94]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12825363 entries, 0 to 12825362
Data columns (total 18 columns):
 #   Column          Dtype         
---  ------          -----         
 0   inventory_id    object        
 1   store_id        float64       
 2   brand_id        float64       
 3   item            object        
 4   size            object        
 5   sales_quantity  int64         
 6   sales_dollars   float64       
 7   sales_price     float64       
 8   sales_date      datetime64[ns]
 9   item_volume     float64       
 10  classification  int64         
 11  excise_tax      float64       
 12  vendor_id       int64         
 13  vendor          object        
 14  month           int64         
 15  dayofweek       int64         
 16  city            object        
 17  volume          float64       
dtypes: datetime64[ns](1), float64(7), int64(5), object(5)
memory usage: 1.7+ GB


### 2-3-2. 가격 정보 칼럼 추가하기

- purchase_price(운송비 미포함 매입단가), retail_price(소매 판매가) 칼럼 추가하기

In [95]:
sales_merged = sales.merge(purchase_prices[['brand_id', 'purchase_price', 'retail_price']], on = 'brand_id', how = 'left')
sales_merged

,inventory_id,store_id,brand_id,item,size,sales_quantity,sales_dollars,sales_price,sales_date,item_volume,classification,excise_tax,vendor_id,vendor,month,dayofweek,city,volume,purchase_price,retail_price
0,1_SPARROW_1004,69.0,8412.0,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,0,SPARROW,750.0,35.71,49.99
1,1_SPARROW_1004,30.0,5255.0,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY,1,1,SPARROW,750.0,9.35,12.99
2,1_SPARROW_1004,34.0,5215.0,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,2,SPARROW,750.0,9.41,12.99
3,1_SPARROW_1004,1.0,5255.0,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,0,SPARROW,750.0,9.35,12.99
4,1_SPARROW_1005,76.0,2034.0,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY,1,1,SPARROW,750.0,21.32,32.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12825358,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-17,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,1,BLUEBIRD,750.0,NaN,NaN
12825359,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-21,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,5,BLUEBIRD,750.0,NaN,NaN
12825360,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,3,77.97,25.99,2024-12-23,750.0,1,2.36,9165,ULTRA BEVERAGE COMPANY LLP,12,0,BLUEBIRD,750.0,NaN,NaN
12825361,9_BLUEBIRD_984,NaN,NaN,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-24,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP,12,1,BLUEBIRD,750.0,NaN,NaN


In [96]:
sales_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12825363 entries, 0 to 12825362
Data columns (total 20 columns):
 #   Column          Dtype         
---  ------          -----         
 0   inventory_id    object        
 1   store_id        float64       
 2   brand_id        float64       
 3   item            object        
 4   size            object        
 5   sales_quantity  int64         
 6   sales_dollars   float64       
 7   sales_price     float64       
 8   sales_date      datetime64[ns]
 9   item_volume     float64       
 10  classification  int64         
 11  excise_tax      float64       
 12  vendor_id       int64         
 13  vendor          object        
 14  month           int64         
 15  dayofweek       int64         
 16  city            object        
 17  volume          float64       
 18  purchase_price  float64       
 19  retail_price    float64       
dtypes: datetime64[ns](1), float64(9), int64(5), object(5)
memory usage: 2.0+ GB


# 3. 데이터 저장

In [97]:
# CSV로 저장
inventory.to_csv('../data/preprocessed/inventory.csv', index=False)
purchases_vendor.to_csv('../data/preprocessed/purchases_vendor.csv', index=False)
sales_merged.to_csv('../data/preprocessed/sales.csv', index=False)
purchase_prices.to_csv('../data/preprocessed/purchase_prices.csv', index=False)